In [5]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
import sklearn
import pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.utils import shuffle
from sklearn.svm import SVC
from itertools import product

pd.options.display.max_rows = 1000

def xgbGridSearchCV(data, parameter, cv = 5):

    grid_param = dict()
    model_param = dict()


    ## grid search combination
    for key,values in  parameter.items():
        if type(values) is list:
            grid_param[key] = values
        else:
            model_param[key] = values

    perm = [dict(zip(grid_param, v)) for v in product(*grid_param.values())]

    result = list()
    
    ## CV
    for item in perm:
        temp = dict()
        temp.update(model_param)
        temp.update(item)
        cv_result = xgb.cv(temp, data, num_boost_round=20000, nfold=5,early_stopping_rounds=100)
        print(cv_result)
        
        
        score = list(cv_result['test-auc-mean'])[-1]
        item.update({"score":score})
        result.append(item)
        
    return result


In [2]:
def cap30(xgbresult):
    
    df = xgbresult[['y_pred','y']].sort_values(by=['y_pred'])
    df = xgbresult[['y_pred','y']].sort_values(by=['y_pred'])
    total_y = (df['y']==1).astype(int).sum()
    num_y = (df.iloc[int(df.shape[0]*0.7):]['y']==1).astype(int).sum()
    
    return num_y/total_y

def cap10(xgbresult):
    
    df = xgbresult[['y_pred','y']].sort_values(by=['y_pred'])
    df = xgbresult[['y_pred','y']].sort_values(by=['y_pred'])
    total_y = (df['y']==1).astype(int).sum()
    num_y = (df.iloc[int(df.shape[0]*0.9):]['y']==1).astype(int).sum()
    
    return num_y/total_y

In [4]:
x_1803 = pd.read_hdf('../group1_1803.h5', key='all', mode='r')
drop = [x for x in x_1803.columns if "age" in x[:4]]
x_1803 = x_1803.drop(columns = drop)
id_1803 = pd.read_hdf('../group1_1803.h5', key='id', mode='r')
y_1803 = pd.read_hdf('../group1_1803.h5', key='y', mode='r')


x_1806 = pd.read_hdf('../group1_1806.h5', key='all', mode='r')
drop = [x for x in x_1806.columns if "age" in x[:4]]
x_1806 = x_1806.drop(columns = drop)
id_1806 = pd.read_hdf('../group1_1806.h5', key='id', mode='r')
y_1806 = pd.read_hdf('../group1_1806.h5', key='y', mode='r')

x_1809 = pd.read_hdf('../group1_1809.h5', key='all', mode='r')
drop = [x for x in x_1809.columns if "age" in x[:4]]
x_1809 = x_1809.drop(columns = drop)
id_1809 = pd.read_hdf('../group1_1809.h5', key='id', mode='r')
y_1809 = pd.read_hdf('../group1_1809.h5', key='y', mode='r')


y_1803 = y_1803.apply(lambda x : 1 if x!=0 else 0)
y_1806 = y_1806.apply(lambda x : 1 if x!=0 else 0)


In [6]:
x_1803 = pd.read_hdf('../group1_1803.h5', key='all', mode='r')

In [7]:
x_1803.shape

(171867, 483)

In [8]:
s_x_1803 = pd.read_hdf('../s1_1803.h5', key='all', mode='r')

In [9]:
s_x_1803.shape

(171867, 684)

In [5]:
s_x_1803 = pd.read_hdf('../s1_1803.h5', key='all', mode='r')
s_id_1803 = pd.read_hdf('../s1_1803.h5', key='id', mode='r')
s_y_1803 = pd.read_hdf('../s1_1803.h5', key='y', mode='r')

s_x_1806 = pd.read_hdf('../s1_1806.h5', key='all', mode='r')
s_id_1806 = pd.read_hdf('../s1_1806.h5', key='id', mode='r')
s_y_1806 = pd.read_hdf('../s1_1806.h5', key='y', mode='r')

s_x_1809 = pd.read_hdf('../s1_1809.h5', key='all', mode='r')
s_id_1809 = pd.read_hdf('../s1_1809.h5', key='id', mode='r')
s_y_1809 = pd.read_hdf('../s1_1809.h5', key='y', mode='r')

In [6]:
x_1803['enc_customer_id'] = id_1803
x_1806['enc_customer_id'] = id_1806
x_1809['enc_customer_id'] = id_1809

s_x_1803['enc_customer_id'] = s_id_1803
s_x_1806['enc_customer_id'] = s_id_1806
s_x_1809['enc_customer_id'] = s_id_1809

In [7]:
x_1803_total = pd.merge(x_1803, s_x_1803, how="left", on='enc_customer_id')
x_1806_total = pd.merge(x_1806, s_x_1806, how="left", on='enc_customer_id')
x_1809_total = pd.merge(x_1809, s_x_1809, how="left", on='enc_customer_id')

In [8]:
x_train = pd.concat([x_1803_total, x_1806_total, x_1809_total], ignore_index=True)
y_train = pd.concat([y_1803, y_1806, s_y_1809], ignore_index=True)
x_train = x_train.drop(columns = ['enc_customer_id'])

/home/aiuser01/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [2]:
feature = ['JCIC_M_BANKCNT_MEAN_L12',
 'ever_mor',
 'ever_submor',
 'JC_224',
 'M_Loan_Amt_MEAN_L12',
 'ctbc_finmortgage_amt_max_3Y',
 'READ_PCA_11_MEAN_L3',
 'SAPBPT',
 'cnt_of_membe',
 'nums_cust_grade_max_3Y_08',
 'Read_hour4_GROW_3M',
 'TAKE_AMT_L1_DP_PERCENT',
 'READ_CLASS_23_MEAN_L3',
 'MEAN_GIVE_LOAN_AMT_L1',
 'READ_CLASS_11_L1',
 'JC_266',
 'M_Loan_Amt_ESTMEAN_L12',
 'JC_244',
 'Click_hour4_MEAN_L3',
 'TARG_Scorecard_Type_Code_TARGTERM',
 'SUM_PERM_LIMIT_GROW_12M',
 'READ_PCA_17_L0',
 'Employment_Year',
 'nums_appl_cnt_10Y',
 'MAX_Retail_Balance_GROW_3M',
 'READ_CLASS_18_GROW_3M',
 'SUM_PERM_LIMIT_NEWBETA_3M',
 'CATMcntD6M',
 'jcic_fm_bankcnt_3Y',
 'JC_199',
 'JC_085',
 'nums_cust_grade_max_3Y_Mis',
 'READ_PCA_17_MEAN_L3',
 'READ_PCA_20_L0',
 'KBamt',
 'JCIC_M_BANKCNT_ESTMEAN_L12',
 'JC_195',
 'jcic_mortgage_amt_sum_3Y',
 'sum_of_house_SPEND_PAY',
 'M_Loan_Amt_BETA_3M',
 'SUM_PERM_LIMIT_MEAN_L3',
 'SUM_OTHER_AMT_NEWBETA_12M',
 'SAPBamt',
 'READ_CLASS_17_GROW_3M',
 'READ_PCA_6_GROW_3M',
 'TAKE_AMT_L3_DP_PERCENT',
 'MIN_JCIC_URATE_EST3M_L0',
 'CLICK_FREQ_MEAN_L3',
 'JC_303',
 'Loan_Amt_BETA_3M']

In [11]:
len(feature)

50

In [16]:
param = {
    'objective': 'binary:logistic',
    'max_depth' : 4,
    'eval_metric':'auc',
    'nthread': 4,
    'eta': 0.1
}

In [32]:
x_train.shape

(524991, 1179)

In [13]:
y_train.shape

(524991,)

In [39]:
x_train_fold, x_test_fold, y_train_fold, y_test_fold = train_test_split(x_train[feature], y_train,test_size = 0.2,stratify = y_train, random_state = 1255)
dtrain = xgb.DMatrix(x_train[feature], y_train)
dtest = xgb.DMatrix(x_test_fold, y_test_fold)

/home/aiuser01/anaconda3/lib/python3.6/site-packages/xgboost-0.83.dev0-py3.6.egg/xgboost/core.py:591: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [40]:
watchlist = [(dtrain, 'train'), (dtest, 'eval')]
model = xgb.train(param, dtrain, evals = watchlist, num_boost_round = 80, early_stopping_rounds = 50)

[0]	train-auc:0.761524	eval-auc:0.767273
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[1]	train-auc:0.766503	eval-auc:0.772352
[2]	train-auc:0.769909	eval-auc:0.777531
[3]	train-auc:0.771439	eval-auc:0.779348
[4]	train-auc:0.771418	eval-auc:0.779339
[5]	train-auc:0.772293	eval-auc:0.779893
[6]	train-auc:0.772665	eval-auc:0.780011
[7]	train-auc:0.77288	eval-auc:0.780129
[8]	train-auc:0.776775	eval-auc:0.786267
[9]	train-auc:0.777471	eval-auc:0.786892
[10]	train-auc:0.777425	eval-auc:0.786857
[11]	train-auc:0.777633	eval-auc:0.787128
[12]	train-auc:0.777895	eval-auc:0.787363
[13]	train-auc:0.778288	eval-auc:0.787622
[14]	train-auc:0.778924	eval-auc:0.787942
[15]	train-auc:0.778876	eval-auc:0.788292
[16]	train-auc:0.78055	eval-auc:0.789805
[17]	train-auc:0.780727	eval-auc:0.789829
[18]	train-auc:0.781057	eval-auc:0.789755
[19]	train-auc:0.781289	eval-auc:0.790132
[20]	train-auc:0.781654	eval-au

In [67]:
# with open("group1_xgb.pkl","wb") as f:
#     pickle.dump(model,f)

In [5]:
with open("group1_xgb.pkl","rb") as f:
    model = pickle.load(f)

In [42]:
x_1712 = pd.read_hdf('../group1_1712.h5', key='all', mode='r')
id_1712 = pd.read_hdf('../group1_1712.h5', key='id', mode='r')
y_1712 = pd.read_hdf('../group1_1712.h5', key='y', mode='r')

y_1712 = y_1712.apply(lambda x : 1 if x!=0 else 0)

s_x_1712 = pd.read_hdf('../s1_1712.h5', key='all', mode='r')
s_id_1712 = pd.read_hdf('../s1_1712.h5', key='id', mode='r')

x_1712['enc_customer_id'] = id_1712
s_x_1712['enc_customer_id'] = s_id_1712
x_1712_total = pd.merge(x_1712, s_x_1712, how="left", on='enc_customer_id')

In [47]:
x_1712_total['READ_PCA_11_MEAN_L3']=0
pred = model.predict(xgb.DMatrix(x_1712_total[feature]))
print(roc_auc_score(y_1712, pred)*2-1)
# d = {'y_binary': y_1803, 'score_100': pred}
# df = pd.DataFrame(data=d)
# print(gini_calculate(df))
d = {'y': y_1712, 'y_pred': pred}
df = pd.DataFrame(data=d)
print("cap30:", cap30(df))

0.6006286861944883
cap30: 0.7608695652173914


In [51]:
x_1803 = pd.read_hdf('../group1_1803.h5', key='all', mode='r')
id_1803 = pd.read_hdf('../group1_1803.h5', key='id', mode='r')
y_1803 = pd.read_hdf('../group1_1803.h5', key='y', mode='r')

y_1803 = y_1803.apply(lambda x : 1 if x!=0 else 0)

s_x_1803 = pd.read_hdf('../s1_1803.h5', key='all', mode='r')
s_id_1803 = pd.read_hdf('../s1_1803.h5', key='id', mode='r')

x_1803['enc_customer_id'] = id_1803
s_x_1803['enc_customer_id'] = s_id_1803
x_1803_total = pd.merge(x_1803, s_x_1803, how="left", on='enc_customer_id')


In [53]:
x_1803_total['READ_PCA_11_MEAN_L3']=0
x_1803_total['READ_CLASS_23_MEAN_L3']=0
x_1803_total['READ_PCA_17_L0']=0
x_1803_total['READ_PCA_17_MEAN_L3']=0
x_1803_total['READ_CLASS_17_GROW_3M']=0

pred = model.predict(xgb.DMatrix(x_1803_total[feature]))
print(roc_auc_score(y_1803, pred)*2-1)
# d = {'y_binary': y_1803, 'score_100': pred}
# df = pd.DataFrame(data=d)
# print(gini_calculate(df))
d = {'y': y_1803, 'y_pred': pred}
df = pd.DataFrame(data=d)
print("cap30:", cap30(df))

0.5959333489537677
cap30: 0.7710894446799491


In [54]:
x_1806 = pd.read_hdf('../group1_1806.h5', key='all', mode='r')
id_1806 = pd.read_hdf('../group1_1806.h5', key='id', mode='r')
y_1806 = pd.read_hdf('../group1_1806.h5', key='y', mode='r')

y_1806 = y_1806.apply(lambda x : 1 if x!=0 else 0)

s_x_1806 = pd.read_hdf('../s1_1806.h5', key='all', mode='r')
s_id_1806 = pd.read_hdf('../s1_1806.h5', key='id', mode='r')

x_1806['enc_customer_id'] = id_1806
s_x_1806['enc_customer_id'] = s_id_1806
x_1806_total = pd.merge(x_1806, s_x_1806, how="left", on='enc_customer_id')

In [57]:
x_1806_total['READ_CLASS_23_MEAN_L3']=0
x_1806_total['READ_CLASS_17_GROW_3M']=0


pred = model.predict(xgb.DMatrix(x_1806_total[feature]))
print(roc_auc_score(y_1806, pred)*2-1)
# d = {'y_binary': y_1803, 'score_100': pred}
# df = pd.DataFrame(data=d)
# print(gini_calculate(df))
d = {'y': y_1806, 'y_pred': pred}
df = pd.DataFrame(data=d)
print("cap30:", cap30(df))

0.5871325301340737
cap30: 0.7579313342025207


In [61]:
x_1809 = pd.read_hdf('../group1_1809.h5', key='all', mode='r')
id_1809 = pd.read_hdf('../group1_1809.h5', key='id', mode='r')
y_1809 = pd.read_hdf('../group1_1809.h5', key='y', mode='r')

y_1809 = y_1809.apply(lambda x : 1 if x!=0 else 0)

s_x_1809 = pd.read_hdf('../s1_1809.h5', key='all', mode='r')
s_id_1809 = pd.read_hdf('../s1_1809.h5', key='id', mode='r')
s_y_1809 = pd.read_hdf('../s1_1809.h5', key='y', mode='r')

x_1809['enc_customer_id'] = id_1809
s_x_1809['enc_customer_id'] = s_id_1809
x_1809_total = pd.merge(x_1809, s_x_1809, how="left", on='enc_customer_id')




In [62]:
x_1809_total['READ_PCA_17_L0']=0
x_1809_total['READ_PCA_6_GROW_3M']=0


pred = model.predict(xgb.DMatrix(x_1809_total[feature]))
print(roc_auc_score(s_y_1809, pred)*2-1)
# d = {'y_binary': y_1803, 'score_100': pred}
# df = pd.DataFrame(data=d)
# print(gini_calculate(df))
d = {'y': s_y_1809, 'y_pred': pred}
df = pd.DataFrame(data=d)
print("cap30:", cap30(df))

0.6012339956954948
cap30: 0.762080073630925


In [63]:
x_1812 = pd.read_hdf('../group1_1812.h5', key='all', mode='r')
id_1812 = pd.read_hdf('../group1_1812.h5', key='id', mode='r')
y_1812 = pd.read_hdf('../group1_1812.h5', key='y', mode='r')

y_1812 = y_1812.apply(lambda x : 1 if x!=0 else 0)

s_x_1812 = pd.read_hdf('../s1_1812.h5', key='all', mode='r')
s_id_1812 = pd.read_hdf('../s1_1812.h5', key='id', mode='r')
s_y_1812 = pd.read_hdf('../s1_1812.h5', key='y', mode='r')

x_1812['enc_customer_id'] = id_1812
s_x_1812['enc_customer_id'] = s_id_1812
x_1812_total = pd.merge(x_1812, s_x_1812, how="left", on='enc_customer_id')


In [66]:
x_1812_total['READ_PCA_17_L0']=0
pred = model.predict(xgb.DMatrix(x_1812_total[feature]))
print(roc_auc_score(s_y_1812, pred)*2-1)
# d = {'y_binary': y_1803, 'score_100': pred}
# df = pd.DataFrame(data=d)
# print(gini_calculate(df))
d = {'y': s_y_1812, 'y_pred': pred}
df = pd.DataFrame(data=d)
print("cap30:", cap30(df))

0.6349131495629499
cap30: 0.8056049822064056


## score table

In [3]:
x_1712 = pd.read_hdf('../group1_1712.h5', key='all', mode='r')
id_1712 = pd.read_hdf('../group1_1712.h5', key='id', mode='r')
y_1712 = pd.read_hdf('../group1_1712.h5', key='y', mode='r')

y_1712 = y_1712.apply(lambda x : 1 if x!=0 else 0)

s_x_1712 = pd.read_hdf('../s1_1712.h5', key='all', mode='r')
s_id_1712 = pd.read_hdf('../s1_1712.h5', key='id', mode='r')

x_1712['enc_customer_id'] = id_1712
s_x_1712['enc_customer_id'] = s_id_1712
x_1712_total = pd.merge(x_1712, s_x_1712, how="left", on='enc_customer_id')

In [6]:
x_1712_total['READ_PCA_11_MEAN_L3']=0
pred = model.predict(xgb.DMatrix(x_1712_total[feature]))

In [7]:
d = {'y': y_1712, 'y_pred': pred}
xgbresult = pd.DataFrame(data=d)
xgbresult = xgbresult.sort_values(by='y_pred',ascending=False).reset_index(drop=True)
xgbresult['score_100'] =  np.round(xgbresult['y_pred']*100)  
scoringtable = xgbresult[['y','score_100']].pivot_table(index='score_100', columns='y', aggfunc=len, fill_value=0)
scoringtable['goodrate'] = scoringtable[1]/(scoringtable[0]+scoringtable[1])
bins = [-0.001, 0.001, 0.002, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06,
           0.07, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.2, 0.25, 0.3, 1.1]
labels = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
scoringtable['score'] = pd.cut(scoringtable['goodrate'], bins=bins, labels=labels)    
scoringtable =scoringtable.reset_index()[['score_100','score']]

In [9]:
scoringtable.to_csv('scoringtable_group1_xgb.csv', index = False)

In [8]:
scoringtable

y,score_100,score
0,0.0,3
1,1.0,4
2,2.0,5
3,3.0,6
4,4.0,8
5,5.0,9
6,6.0,9
7,7.0,12
8,8.0,12
9,9.0,12


In [10]:
y_1712.value_counts()

0    166316
1      2024
Name: SEC_Y_3M, dtype: int64